# Routing and avoiding construction sites

In this example, we'd like to showchase how to use the [**direction API**](https://openrouteservice.org/documentation/#/reference/directions/directions/directions-service) and to avoid a number of polygons while routing. The challenge here is you can just avoid polygons, not points and how to handle with a high number of areas which are to avoid. 

In [1]:
import folium

import json

from openrouteservice import client

from shapely import geometry
from shapely.geometry import Point, LineString

Rostock is beautiful, but like in every other city there are so many construction sites. Wouldn't it be great if we'd already know straight away which way to go without crossing any working areas?!

## Rostock construction sites

We have a json data file with all construction sites in Rostock saved as points. To start we load in the data and visualize them on a folium map.

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # Individual api key
clnt = client.Client(key=api_key) # Create client with api key
rostock_map = folium.Map(tiles='Stamen Toner', location=([54.13207, 12.101612]), zoom_start=12) 

with open('baustellen_rostock.json') as data: # Load in construction data points
    rostock_json = json.load(data)

# Create map with all construction points
construction_points = []
for point_data in rostock_json['features']:
    data_coords = point_data['geometry']['coordinates']
    folium.features.Marker(list(reversed(data_coords)), popup='Construction point<br>{0}'.format(data_coords)).add_to(rostock_map)
    construction_points.append(data_coords)
    
rostock_map

Now the challenging part: The GET direction service can just work with a limited number of characters and not with the high number of construction sites, why the request here should always just avoid the working areas which are in the immediate surroundings to keep it as small as possible. 
Therefor two functions will be defined, one for the routing request with the specific avoided construction sites and one to create a street buffer to check which working site has to be avoid. 

In [3]:
# Function to request directions 
def create_route(avoided_point_list, n=0):
    route_request = {'coordinates': [start, destination], 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                      'instructions': False,
                     'options': {'avoid_polygons': {'type': 'MultiPolygon', 'coordinates': avoided_point_list}}
                     }
    route_directions = clnt.directions(**route_request)
    
    return route_directions

# Function to create buffer around requested route
def create_buffer(route_directions):
    line_tup = []
    for line in route_directions['features'][0]['geometry']['coordinates']:
        tup_format = tuple(line)
        line_tup.append(tup_format)  

    new_linestring = LineString(line_tup)
    dilated_route = new_linestring.buffer(0.001)
        
    return dilated_route

The Shapely package is now very useful to check if one point geometry lays within the street buffer geometry. If yes a small polygon will be created out of the point and pushed into the direction request. The data for the request is needed as strings in a listed list. 
This process will repeat until there is a route without a point within the buffer and on the street. 

If there is no route available an error and a warning will appear.

In [4]:
start = [12.088019, 54.084592]
destination = [12.131739, 54.082398] # First example: Route available.
#destination = [12.129453, 54.082845] # Second example: No route available because of too many construction sites.

# Request route without avoiding any construction points
avoided_point_list = []
route_directions = create_route(avoided_point_list) 
dilated_route = create_buffer(route_directions)
    
# Create geometry of construction points 
point_geom = []
for point in construction_points:
    poi_tup = tuple(point)
    poi_geom = geometry.Point(poi_tup)
    point_geom.append(poi_geom)

try:
    # Request route with avoiding construction points which are located on route    
    for point in point_geom:
        if point.within(dilated_route): 
            point_coords = list(point.coords) # get point coordinates

            # Create avoiding area around point
            avoid_box = []
            t = 0.0003
            for x,y in point_coords: 
                point_area = [[x-t, y-t], [x+t, y-t], [x+t, y+t], [x-t, y+t], [x-t, y-t]]
                avoid_box.append(point_area) 

            # Data is needed as strings in listed lists 
            poly_coords_list = [] # Coords of each polygon -> will be deleted after use
            for poly in avoid_box:
                for row in poly:
                    coords = list(row)
                    i = 0
                    for l in coords:
                        i = i+1
                        if i%2 != 0:
                            x = l
                        if i%2 == 0:
                            y = l

                            kombi_coords = [str(x), str(y)]
                            poly_coords_list.append(kombi_coords)

                avoided_point_list.append([poly_coords_list]) # Coords of all related polygons
                poly_coords_list = []

            # Create new route and buffer
            route_directions = create_route(avoided_point_list, 1)
            dilated_route = create_buffer(route_directions)
        
except Exception: print('Sorry, there is no route available to the requested destination because of too may construction sites. Please choose a nearby location for a new routing request or park the car in the aim area and walk the last part.')    

route_coords = [(y,x) for x,y in route_directions['features'][0]['geometry']['coordinates']]
folium.PolyLine(route_coords, color='red').add_to(rostock_map)

folium.map.Marker(list(reversed(start)), folium.Icon(color='green')).add_to(rostock_map)
folium.map.Marker(list(reversed(destination)), folium.Icon(color='green')).add_to(rostock_map)

rostock_map

That is great! Finally a routing without the annoying construction sites and still the shortest way to go.